In [1]:
#real time capture
import cv2
import dlib
import numpy as np
import tensorflow as tf
import time

# Load trained model
model = tf.keras.models.load_model("eye_state_vgg_small_eye_dataset.h5")

# Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Dlib shape predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Eye landmark indices (68-point model)
LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))

# Video capture
cap = cv2.VideoCapture(0)

closed_start_time = None
drowsy_alert = False
threshold_time = 2  # seconds

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        cv2.putText(frame, "Unknown Face", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        closed_start_time = None
        drowsy_alert = False
    else:
        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            # Convert to dlib rectangle
            dlib_rect = dlib.rectangle(0, 0, w, h)
            landmarks = predictor(roi_gray, dlib_rect)

            def extract_eye(points):
                coords = np.array([(landmarks.part(p).x, landmarks.part(p).y) for p in points])
                ex, ey, ew, eh = cv2.boundingRect(coords)
                eye = roi_color[ey:ey+eh, ex:ex+ew]
                if eye.size == 0:
                    return None, (ex,ey,ew,eh)
                eye_resized = cv2.resize(eye, (224,224))
                eye_normalized = eye_resized / 255.0
                return np.expand_dims(eye_normalized, axis=0), (ex,ey,ew,eh)

            left_eye, boxL = extract_eye(LEFT_EYE_POINTS)
            right_eye, boxR = extract_eye(RIGHT_EYE_POINTS)

            eye_states = []

            for eye_input, box in [(left_eye, boxL), (right_eye, boxR)]:
                if eye_input is not None:
                    pred = model.predict(eye_input, verbose=0)[0][0]
                    state = "Closed" if pred < 0.5 else "Open"  # adjust based on class_indices
                    eye_states.append(state)

                    (ex,ey,ew,eh) = box
                    color = (0,255,0) if state=="Open" else (0,0,255)
                    cv2.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), color, 2)
                    cv2.putText(frame, state, (ex,ey-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Check drowsiness
            if len(eye_states) > 0 and all(s=="Closed" for s in eye_states):
                if closed_start_time is None:
                    closed_start_time = time.time()
                else:
                    elapsed = time.time() - closed_start_time
                    if elapsed > threshold_time:
                        drowsy_alert = True
            else:
                closed_start_time = None
                drowsy_alert = False

            if drowsy_alert:
                cv2.putText(frame, "DROWSINESS ALERT!", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 3)

    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#image capture

import cv2
import dlib
import numpy as np
import tensorflow as tf

# Load trained model
model = tf.keras.models.load_model("eye_state_vgg_small_eye_dataset.h5")

# Dlib face detector + shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  


# Landmark indices for eyes (68-point model)
LEFT_EYE_POINTS = list(range(36, 42))   # left eye
RIGHT_EYE_POINTS = list(range(42, 48))  # right eye

# Path to test image
image_path = "A0787.png"

# Read image
frame = cv2.imread(image_path)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

faces = detector(gray)

if len(faces) == 0:
    print("Unknown Face")
else:
    for face in faces:
        landmarks = predictor(gray, face)

        def extract_eye(points):
            coords = np.array([(landmarks.part(p).x, landmarks.part(p).y) for p in points])
            x, y, w, h = cv2.boundingRect(coords)
            eye = frame[y:y+h, x:x+w]
            if eye.size == 0:
                return None
            eye_resized = cv2.resize(eye, (224,224))
            eye_normalized = eye_resized / 255.0
            return np.expand_dims(eye_normalized, axis=0), (x,y,w,h)

        # Left eye
        left_eye, boxL = extract_eye(LEFT_EYE_POINTS)
        # Right eye
        right_eye, boxR = extract_eye(RIGHT_EYE_POINTS)

        eye_states = []

        for eye_input, box in [(left_eye, boxL), (right_eye, boxR)]:
            if eye_input is not None:
                pred = model.predict(eye_input, verbose=0)[0][0]

                #  adjust based on class_indices
                state = "Closed" if pred < 0.5 else "Open"
                eye_states.append(state)

                (x,y,w,h) = box
                color = (0,255,0) if state=="Open" else (0,0,255)
                cv2.rectangle(frame, (x,y), (x+w,y+h), color, 2)
                cv2.putText(frame, state, (x,y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        if len(eye_states) > 0:
            print("Detected:", eye_states)

# Show result
cv2.imshow("Result", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()



Detected: ['Closed', 'Closed']


In [ ]:
#EYE METRICS 
import cv2
import dlib
import numpy as np
import tensorflow as tf
import time
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ------------------------------
# 🔹 1. Load trained model
# ------------------------------
model = tf.keras.models.load_model("eye_state_vgg_small_eye_dataset.h5")

# ------------------------------
# 🔹 2. Evaluate on test dataset
# ------------------------------
test_dir = r"C:\Users\gupta\OneDrive\Desktop\computer vision project\myeyes_2"  # path to your test dataset

test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)

# Show detected classes and folders
print("Detected classes:", test_generator.class_indices)
print("Folders in test dir:", os.listdir(test_dir))

# Predict
y_pred = model.predict(test_generator, verbose=1)
y_pred = (y_pred > 0.5).astype(int).flatten()
y_true = test_generator.classes

# Define labels (Closed/Open)
labels = [0, 1]
class_names = ["closed", "open"]

# Evaluation
print("\n✅ Test Accuracy:", accuracy_score(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred, labels=labels))
print("\nClassification Report:\n",classification_report(y_true, y_pred, labels=labels, target_names=class_names))



# Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Dlib shape predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Eye landmark indices
LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))

# Video capture
cap = cv2.VideoCapture(0)

closed_start_time = None
drowsy_alert = False
threshold_time = 2        # seconds
closed_threshold = 0.45   # lowered threshold for Closed detection

print("\n Starting Real-time Drowsiness Detection... Press ESC to exit.\n")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        cv2.putText(frame, "Unknown Face", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        closed_start_time = None
        drowsy_alert = False
    else:
        for (x, y, w, h) in faces:
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]

            dlib_rect = dlib.rectangle(0, 0, w, h)
            landmarks = predictor(roi_gray, dlib_rect)

            def extract_eye(points):
                coords = np.array([(landmarks.part(p).x, landmarks.part(p).y) for p in points])
                ex, ey, ew, eh = cv2.boundingRect(coords)
                eye = roi_color[ey:ey + eh, ex:ex + ew]
                if eye.size == 0:
                    return None, (ex, ey, ew, eh)
                eye_resized = cv2.resize(eye, (224, 224))
                eye_normalized = eye_resized / 255.0
                return np.expand_dims(eye_normalized, axis=0), (ex, ey, ew, eh)

            left_eye, boxL = extract_eye(LEFT_EYE_POINTS)
            right_eye, boxR = extract_eye(RIGHT_EYE_POINTS)

            eye_states = []

            for eye_input, box in [(left_eye, boxL), (right_eye, boxR)]:
                if eye_input is not None:
                    pred = model.predict(eye_input, verbose=0)[0][0]
                    state = "Closed" if pred < closed_threshold else "Open"
                    eye_states.append(state)

                    (ex, ey, ew, eh) = box
                    color = (0, 255, 0) if state == "Open" else (0, 0, 255)
                    cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), color, 2)
                    cv2.putText(frame, state, (ex, ey - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

                    # DEBUG: print prediction value
                    print(f"Eye prediction: {pred:.3f} -> {state}")

            # Drowsiness check
            if len(eye_states) > 0 and all(s == "Closed" for s in eye_states):
                if closed_start_time is None:
                    closed_start_time = time.time()
                else:
                    elapsed = time.time() - closed_start_time
                    if elapsed > threshold_time:
                        drowsy_alert = True
            else:
                closed_start_time = None
                drowsy_alert = False

            if drowsy_alert:
                cv2.putText(frame, "DROWSINESS ALERT!", (100, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    cv2.imshow("Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'sklearn'

In [1]:
#eye plus yawn metrics
import cv2
import dlib
import numpy as np
import tensorflow as tf
import time
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator


eye_model = tf.keras.models.load_model("eye_state_vgg_small_eye_dataset.h5")
yawn_model = tf.keras.models.load_model("yawn_state_model.h5")  

#evaluate eye 
eye_test_dir = r"C:\Users\gupta\OneDrive\Desktop\computer vision project\myeyes_2"

test_datagen = ImageDataGenerator(rescale=1.0 / 255)
eye_test_gen = test_datagen.flow_from_directory(
    eye_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)

print("\n🔹 EYE MODEL CLASSES:", eye_test_gen.class_indices)
print("🔹 Eye Test Folders:", os.listdir(eye_test_dir))

eye_pred = eye_model.predict(eye_test_gen, verbose=1)
eye_pred = (eye_pred > 0.5).astype(int).flatten()
eye_true = eye_test_gen.classes

eye_labels = [0, 1]
eye_class_names = ["Closed_Eyes", "Open_Eyes"]

print("\nEye Model Accuracy:", accuracy_score(eye_true, eye_pred))
print("\nConfusion Matrix (Eyes):\n", confusion_matrix(eye_true, eye_pred, labels=eye_labels))
print("\nClassification Report (Eyes):\n", classification_report(eye_true, eye_pred, labels=eye_labels, target_names=eye_class_names))

#evaluate yawn
yawn_test_dir = r"C:\Users\gupta\OneDrive\Desktop\computer vision project\my_yawn" 

yawn_test_gen = test_datagen.flow_from_directory(
    yawn_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)

print("\n YAWN MODEL CLASSES:", yawn_test_gen.class_indices)
print(" Yawn Test Folders:", os.listdir(yawn_test_dir))

# Predict for yawn model
yawn_pred = yawn_model.predict(yawn_test_gen, verbose=1)
yawn_pred = (yawn_pred > 0.5).astype(int).flatten()
yawn_true = yawn_test_gen.classes

yawn_labels = [0, 1]
yawn_class_names = ["No_yawn", "Yawn"]

print("\nYawn Model Accuracy:", accuracy_score(yawn_true, yawn_pred))
print("\nConfusion Matrix (Yawn):\n", confusion_matrix(yawn_true, yawn_pred, labels=yawn_labels))
print("\nClassification Report (Yawn):\n", classification_report(yawn_true, yawn_pred, labels=yawn_labels, target_names=yawn_class_names))


# Real-time Drowsiness Detection

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))
MOUTH_POINTS = list(range(48, 68))

cap = cv2.VideoCapture(0)

closed_start_time = None
drowsy_alert = False
threshold_time = 2
closed_threshold = 0.45
yawn_threshold = 0.35

print("\nStarting Real-time Drowsiness + Yawn Detection... Press ESC to exit.\n")

def extract_eye(points, landmarks, roi_color):
    coords = np.array([(landmarks.part(p).x, landmarks.part(p).y) for p in points])
    ex, ey, ew, eh = cv2.boundingRect(coords)
    eye = roi_color[ey:ey + eh, ex:ex + ew]
    if eye.size == 0:
        return None, (ex, ey, ew, eh)
    eye_resized = cv2.resize(eye, (224, 224))
    eye_normalized = eye_resized / 255.0
    return np.expand_dims(eye_normalized, axis=0), (ex, ey, ew, eh)

def extract_mouth(points, landmarks, roi_color):
    coords = np.array([(landmarks.part(p).x, landmarks.part(p).y) for p in points])
    mx, my, mw, mh = cv2.boundingRect(coords)
    padding = 10
    mx, my = max(mx - padding, 0), max(my - padding, 0)
    mw = min(mw + 2 * padding, roi_color.shape[1] - mx)
    mh = min(mh + 2 * padding, roi_color.shape[0] - my)
    mouth = roi_color[my:my + mh, mx:mx + mw]
    if mouth.size == 0:
        return None, (mx, my, mw, mh)
    mouth_resized = cv2.resize(mouth, (224, 224))
    mouth_resized = cv2.cvtColor(mouth_resized, cv2.COLOR_BGR2RGB)
    mouth_normalized = mouth_resized / 255.0
    return np.expand_dims(mouth_normalized, axis=0), (mx, my, mw, mh)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        cv2.putText(frame, "No Face Detected", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        closed_start_time = None
        drowsy_alert = False
    else:
        for (x, y, w, h) in faces:
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]
            dlib_rect = dlib.rectangle(0, 0, w, h)
            landmarks = predictor(roi_gray, dlib_rect)

            # Eye detection 
            left_eye, boxL = extract_eye(LEFT_EYE_POINTS, landmarks, roi_color)
            right_eye, boxR = extract_eye(RIGHT_EYE_POINTS, landmarks, roi_color)
            eye_states = []

            for eye_input, box in [(left_eye, boxL), (right_eye, boxR)]:
                if eye_input is not None:
                    pred = eye_model.predict(eye_input, verbose=0)[0][0]
                    state = "Closed" if pred < closed_threshold else "Open"
                    eye_states.append(state)
                    (ex, ey, ew, eh) = box
                    color = (0, 255, 0) if state == "Open" else (0, 0, 255)
                    cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), color, 2)
                    cv2.putText(frame, state, (x + ex, y + ey - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            #  Yawn detection 
            mouth_input, mouth_box = extract_mouth(MOUTH_POINTS, landmarks, roi_color)
            yawn_detected = False
            if mouth_input is not None:
                y_pred = yawn_model.predict(mouth_input, verbose=0)[0][0]
                yawn_detected = y_pred > yawn_threshold
                (mx, my, mw, mh) = mouth_box
                color = (0, 0, 255) if yawn_detected else (0, 255, 0)
                cv2.rectangle(roi_color, (mx, my), (mx + mw, my + mh), color, 2)
                cv2.putText(frame, f"Yawn: {y_pred:.2f}", (x + mx, y + my - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

            # Drowsiness check
            eyes_closed = len(eye_states) > 0 and all(s == "Closed" for s in eye_states)
            if eyes_closed or yawn_detected:
                if closed_start_time is None:
                    closed_start_time = time.time()
                else:
                    elapsed = time.time() - closed_start_time
                    if elapsed > threshold_time:
                        drowsy_alert = True
            else:
                closed_start_time = None
                drowsy_alert = False

            if drowsy_alert:
                cv2.putText(frame, " DROWSINESS ALERT!", (100, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Drowsiness + Yawn Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()


Found 315 images belonging to 2 classes.


c:\Users\gupta\OneDrive\Desktop\computer vision project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



🔹 EYE MODEL CLASSES: {'closed': 0, 'open': 1}
🔹 Eye Test Folders: ['closed', 'open']
10/10 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step

Eye Model Accuracy: 0.9015873015873016

Confusion Matrix (Eyes):
 [[116  18]
 [ 13 168]]

Classification Report (Eyes):
               precision    recall  f1-score   support

 Closed_Eyes       0.90      0.87      0.88       134
   Open_Eyes       0.90      0.93      0.92       181

    accuracy                           0.90       315
   macro avg       0.90      0.90      0.90       315
weighted avg       0.90      0.90      0.90       315

Found 630 images belonging to 2 classes.

 YAWN MODEL CLASSES: {'no_yawn': 0, 'yawn': 1}
 Yawn Test Folders: ['no_yawn', 'yawn']


c:\Users\gupta\OneDrive\Desktop\computer vision project\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step

Yawn Model Accuracy: 0.8984126984126984

Confusion Matrix (Yawn):
 [[270   3]
 [ 61 296]]

Classification Report (Yawn):
               precision    recall  f1-score   support

     No_yawn       0.82      0.99      0.89       273
        Yawn       0.99      0.83      0.90       357

    accuracy                           0.90       630
   macro avg       0.90      0.91      0.90       630
weighted avg       0.91      0.90      0.90       630


Starting Real-time Drowsiness + Yawn Detection... Press ESC to exit.

